In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from plotly.subplots import make_subplots


In [ ]:
df = pd.read_csv('continent_clean.csv')

df['Year'] = pd.to_datetime(df['Year'], format='%Y')
df['Stability'] = df['Total'].apply(lambda x : 1 if x > 80 else (-1 if x < 40 else 0))
features = ['C1: Security Apparatus', 'C2: Factionalized Elites',
       'C3: Group Grievance', 'E1: Economy', 'E2: Economic Inequality',
       'E3: Human Flight and Brain Drain', 'P1: State Legitimacy',
       'P2: Public Services', 'P3: Human Rights', 'S1: Demographic Pressures',
       'S2: Refugees and IDPs', 'X1: External Intervention']
label = 'Stability'

# Train-Test Split
train_mask = df['Year'] <= '2020-01-01'
train_df = df[train_mask]
test_df = df[~train_mask]

train_df

In [ ]:
# Model Training
X_train, y_train = train_df[features], train_df[label]
X_test, y_test = test_df[features], test_df[label]

# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Model Prediction
y_pred = model.predict(X_test)

# Model Evaluation
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_rep)

# Feature Importance Visualization
feature_importance = pd.DataFrame({'Feature': features, 'Importance': model.feature_importances_})
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.title('Feature Importance')
plt.xlabel('Importance')
plt.show()

In [ ]:
# Most important features as data
data = df[['C1: Security Apparatus', 'P1: State Legitimacy', 'P2: Public Services', 'S1: Demographic Pressures']]

# Create Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    # Multi-dropdown for country selection
    dcc.Dropdown(
        id='country-selector',
        options=[{'label': country, 'value': country} for country in df['Country'].unique()],
        value=['Country A'],  
        multi=True,
        style={'width': '50%'}
    ),
    # Line plots for selected countries
    dcc.Graph(id='line-plots')
])

# Callback to update line plots based on selected countries
@app.callback(
    Output('line-plots', 'figure'),
    [Input('country-selector', 'value')]
)
def update_line_plots(selected_countries):
    filtered_data = df[df['Country'].isin(selected_countries)]

    # Create subplots for each column
    fig = make_subplots(rows=2, cols=2, subplot_titles=data.columns)

    for i, column in enumerate(data.columns):
        row = (i // 2) + 1
        col = (i % 2) + 1

        for country in selected_countries:
            trace = go.Scatter(
                x=filtered_data[filtered_data['Country'] == country]['Year'],
                y=filtered_data[filtered_data['Country'] == country][column],
                mode='lines',
                name=country
            )
            fig.add_trace(trace, row=row, col=col)
            
        fig.update_yaxes(range=[0, 10], row=row, col=col)

    fig.update_layout(
        height=800,
        showlegend=False,
        title_text='Line Plots for Selected Countries Over Years',
        title_x=0.5,
        title_font=dict(size=16),
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [ ]:
df_tran = df[['Country', 'Year', 'Stability', 'Continent']]
df_tran['Year'] = pd.to_datetime(df_tran['Year']).dt.year

# Create Dash app
app = dash.Dash(__name__)

# Layout of the app
app.layout = html.Div([
    # Dropdown for selecting continent
    dcc.Dropdown(
        id='continent-selector',
        options=[{'label': continent, 'value': continent} for continent in df_tran['Continent'].unique()],
        value=df_tran['Continent'].unique()[0], 
        style={'width': '50%'}
    ),
    # Heatmap for stability based on continent
    dcc.Graph(id='stability-heatmap')
])

# Callback to update stability heatmap based on selected continent
@app.callback(
    Output('stability-heatmap', 'figure'),
    [Input('continent-selector', 'value')]
)

def update_heatmap(selected_continent):
    
    # Filter stability data based on selected continent
    filtered_stability = df_tran[df_tran['Continent'] == selected_continent]

    # Randomly select 10 countries
    random_10_countries = filtered_stability['Country'].unique().tolist()
    if len(random_10_countries) > 10:
        random_10_countries = pd.Series(random_10_countries).sample(10).tolist()

    # Filter stability data for the randomly selected 10 countries
    filtered_stability = df_tran[(df_tran['Continent'] == selected_continent) & (df_tran['Country'].isin(random_10_countries))]
    # Create heatmap using plotly.graph_objs
    heatmap_data = go.Heatmap(
        z=filtered_stability.pivot_table(index='Country', columns='Year', values='Stability').values,
        x=sorted(filtered_stability['Year'].unique()),
        y=sorted(filtered_stability['Country'].unique()),
        colorscale='blues',
        colorbar=dict(title='Stability', tickvals=[-1, 0, 1], ticktext=['-1', '0', '1']),
        zmin=-1,
        zmax=1
    )

    fig_heatmap = go.Figure(data=[heatmap_data])
    fig_heatmap.update_layout(
        title=f'Stability Changes Over the Years for {selected_continent} Countries',
        xaxis=dict(title='Year'),
        yaxis=dict(title='Country')
    )

    return fig_heatmap

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)